# 📊 Exploration des Données Crypto
## Analyse des Features pour Machine Learning

**Date:** 12 novembre 2025  
**Auteur:** Karim Assi  
**Objectif:** Analyser les 54 features créées et préparer pour le ML

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports réussis")

✅ Imports réussis


## 1. Chargement des Données

In [2]:
# Trouver tous les fichiers de features
feature_files = glob.glob('../data/processed/*_features_*.csv')

print(f"📁 Fichiers trouvés: {len(feature_files)}")
for f in feature_files:
    print(f"   - {os.path.basename(f)}")

📁 Fichiers trouvés: 5
   - binancecoin_features_2025-11-12_13-18-26.csv
   - bitcoin_features_2025-11-12_13-18-27.csv
   - cardano_features_2025-11-12_13-18-27.csv
   - ethereum_features_2025-11-12_13-18-27.csv
   - solana_features_2025-11-12_13-18-27.csv


In [3]:
# Charger Bitcoin (notre crypto principale)
btc_file = [f for f in feature_files if 'bitcoin' in f.lower()][0]
df_btc = pd.read_csv(btc_file)

print(f"📊 Bitcoin Features")
print(f"   Shape: {df_btc.shape}")
print(f"   Colonnes: {df_btc.shape[1]}")
print(f"\n🔍 Aperçu:")
df_btc.head()

📊 Bitcoin Features
   Shape: (1, 54)
   Colonnes: 54

🔍 Aperçu:


,timestamp,symbol,price_usd,market_cap_usd,volume_24h_usd,price_change_24h_percent,last_updated,sma_7,sma_30,ema_12,ema_26,rsi_14,macd,macd_signal,macd_histogram,bb_upper,bb_middle,bb_lower,bb_width,price_change_1d,price_change_7d,price_change_30d,volume_ma_7,volume_ma_30,volume_ratio,volume_change_1d,volume_spike,price_to_sma7_ratio,price_to_sma30_ratio,sma_crossover,year,month,day,day_of_week,hour,quarter,is_weekend,is_month_start,is_month_end,fear_greed_index,fg_ma_7,fg_ma_30,fg_change_7d,is_extreme_fear,is_fear,is_neutral,is_greed,is_extreme_greed,price_usd_lag_1d,price_usd_lag_7d,price_usd_lag_30d,volume_24h_usd_lag_1d,volume_24h_usd_lag_7d,volume_24h_usd_lag_30d
0,2025-11-12 11:31:26.683487,BITCOIN,104735.00,2089170460645.28,63767851128.36,-0.35,2025-11-12 11:31:21,104735.00,104735.00,104735.00,104735.00,50.00,0.00,0.00,0.00,NaN,104735.00,NaN,NaN,NaN,NaN,NaN,63767851128.36,63767851128.36,1.00,NaN,0,1.00,1.00,0,2025,11,12,2,11,4,0,0,0,24,24.00,24.00,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Charger toutes les cryptos
cryptos = {}

for file in feature_files:
    basename = os.path.basename(file)
    crypto = basename.split('_features_')[0].upper()
    cryptos[crypto] = pd.read_csv(file)
    print(f"✅ {crypto}: {cryptos[crypto].shape}")

print(f"\n📊 Total: {len(cryptos)} cryptos chargées")

✅ BINANCECOIN: (1, 54)
✅ BITCOIN: (1, 54)
✅ CARDANO: (1, 54)
✅ ETHEREUM: (1, 54)
✅ SOLANA: (1, 54)

📊 Total: 5 cryptos chargées


## 2. Structure des Données

In [5]:
# Informations générales
print("📋 Informations sur les données Bitcoin:")
print(df_btc.info())

📋 Informations sur les données Bitcoin:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 1 non-null      object 
 1   symbol                    1 non-null      object 
 2   price_usd                 1 non-null      float64
 3   market_cap_usd            1 non-null      float64
 4   volume_24h_usd            1 non-null      float64
 5   price_change_24h_percent  1 non-null      float64
 6   last_updated              1 non-null      object 
 7   sma_7                     1 non-null      float64
 8   sma_30                    1 non-null      float64
 9   ema_12                    1 non-null      float64
 10  ema_26                    1 non-null      float64
 11  rsi_14                    1 non-null      float64
 12  macd                      1 non-null      float64
 13  macd_signal               1 n

In [6]:
# Catégoriser les colonnes
categories = {
    'Prix & Volume': [c for c in df_btc.columns if any(x in c.lower() for x in ['price', 'volume', 'market'])],
    'Indicateurs Tech': [c for c in df_btc.columns if any(x in c.lower() for x in ['sma', 'ema', 'rsi', 'macd', 'bb'])],
    'Temporel': [c for c in df_btc.columns if any(x in c.lower() for x in ['year', 'month', 'day', 'hour', 'weekend', 'quarter'])],
    'Sentiment': [c for c in df_btc.columns if any(x in c.lower() for x in ['fear', 'greed', 'fg'])],
    'Lag': [c for c in df_btc.columns if 'lag' in c.lower()],
}

# Colonnes non catégorisées
all_categorized = sum(categories.values(), [])
categories['Autres'] = [c for c in df_btc.columns if c not in all_categorized]

print("📊 Répartition des Features:\n")
for cat, cols in categories.items():
    print(f"{cat}: {len(cols)} features")
    if len(cols) <= 10:
        for col in cols:
            print(f"   • {col}")
    else:
        for col in cols[:5]:
            print(f"   • {col}")
        print(f"   ... et {len(cols)-5} autres")
    print()

📊 Répartition des Features:

Prix & Volume: 20 features
   • price_usd
   • market_cap_usd
   • volume_24h_usd
   • price_change_24h_percent
   • price_change_1d
   ... et 15 autres

Indicateurs Tech: 15 features
   • sma_7
   • sma_30
   • ema_12
   • ema_26
   • rsi_14
   ... et 10 autres

Temporel: 9 features
   • year
   • month
   • day
   • day_of_week
   • hour
   • quarter
   • is_weekend
   • is_month_start
   • is_month_end

Sentiment: 8 features
   • fear_greed_index
   • fg_ma_7
   • fg_ma_30
   • fg_change_7d
   • is_extreme_fear
   • is_fear
   • is_greed
   • is_extreme_greed

Lag: 6 features
   • price_usd_lag_1d
   • price_usd_lag_7d
   • price_usd_lag_30d
   • volume_24h_usd_lag_1d
   • volume_24h_usd_lag_7d
   • volume_24h_usd_lag_30d

Autres: 4 features
   • timestamp
   • symbol
   • last_updated
   • is_neutral



## 3. Analyse des Indicateurs Techniques

In [7]:
# RSI Analysis
print("📈 RSI (Relative Strength Index)\n")
print("="*60)

for crypto, df in cryptos.items():
    if 'rsi_14' in df.columns:
        rsi = df['rsi_14'].iloc[0]
        
        if rsi < 30:
            signal = "🟢 SUR-VENDU (opportunité achat)"
        elif rsi > 70:
            signal = "🔴 SUR-ACHETÉ (risque correction)"
        else:
            signal = "🟡 NEUTRE"
        
        print(f"{crypto:12} | RSI: {rsi:5.1f} | {signal}")

📈 RSI (Relative Strength Index)

BINANCECOIN  | RSI:  50.0 | 🟡 NEUTRE
BITCOIN      | RSI:  50.0 | 🟡 NEUTRE
CARDANO      | RSI:  50.0 | 🟡 NEUTRE
ETHEREUM     | RSI:  50.0 | 🟡 NEUTRE
SOLANA       | RSI:  50.0 | 🟡 NEUTRE


In [8]:
# MACD Analysis
print("\n📊 MACD (Moving Average Convergence Divergence)\n")
print("="*60)

for crypto, df in cryptos.items():
    if all(c in df.columns for c in ['macd', 'macd_signal', 'macd_histogram']):
        macd = df['macd'].iloc[0]
        signal = df['macd_signal'].iloc[0]
        hist = df['macd_histogram'].iloc[0]
        
        trend = "🟢 HAUSSIER" if hist > 0 else "🔴 BAISSIER"
        
        print(f"{crypto:12} | MACD: {macd:7.2f} | Signal: {signal:7.2f} | Hist: {hist:7.2f} | {trend}")


📊 MACD (Moving Average Convergence Divergence)

BINANCECOIN  | MACD:    0.00 | Signal:    0.00 | Hist:    0.00 | 🔴 BAISSIER
BITCOIN      | MACD:    0.00 | Signal:    0.00 | Hist:    0.00 | 🔴 BAISSIER
CARDANO      | MACD:    0.00 | Signal:    0.00 | Hist:    0.00 | 🔴 BAISSIER
ETHEREUM     | MACD:    0.00 | Signal:    0.00 | Hist:    0.00 | 🔴 BAISSIER
SOLANA       | MACD:    0.00 | Signal:    0.00 | Hist:    0.00 | 🔴 BAISSIER


In [9]:
# SMA Crossover Analysis
print("\n📉 SMA Crossover (Golden/Death Cross)\n")
print("="*60)

for crypto, df in cryptos.items():
    if all(c in df.columns for c in ['price_usd', 'sma_7', 'sma_30', 'sma_crossover']):
        price = df['price_usd'].iloc[0]
        sma7 = df['sma_7'].iloc[0]
        sma30 = df['sma_30'].iloc[0]
        crossover = df['sma_crossover'].iloc[0]
        
        if crossover == 1:
            signal = "✅ GOLDEN CROSS (SMA7 > SMA30) - Tendance haussière"
        else:
            signal = "❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière"
        
        print(f"{crypto:12} | Prix: ${price:,.2f} | SMA7: ${sma7:,.2f} | SMA30: ${sma30:,.2f}")
        print(f"             {signal}\n")


📉 SMA Crossover (Golden/Death Cross)

BINANCECOIN  | Prix: $969.47 | SMA7: $969.47 | SMA30: $969.47
             ❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière

BITCOIN      | Prix: $104,735.00 | SMA7: $104,735.00 | SMA30: $104,735.00
             ❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière

CARDANO      | Prix: $0.57 | SMA7: $0.57 | SMA30: $0.57
             ❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière

ETHEREUM     | Prix: $3,515.62 | SMA7: $3,515.62 | SMA30: $3,515.62
             ❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière

SOLANA       | Prix: $159.19 | SMA7: $159.19 | SMA30: $159.19
             ❌ DEATH CROSS (SMA7 < SMA30) - Tendance baissière



## 4. Analyse du Sentiment (Fear & Greed)

In [10]:
# Fear & Greed Analysis
print("😱 Fear & Greed Index Analysis\n")
print("="*60)

for crypto, df in cryptos.items():
    if 'fear_greed_index' in df.columns:
        fg = df['fear_greed_index'].iloc[0]
        
        if fg <= 25:
            classification = "😱 EXTREME FEAR"
            action = "🟢 Opportunité d'ACHAT (contrarian)"
        elif fg <= 45:
            classification = "😰 FEAR"
            action = "🟡 Prudent pour ACHAT"
        elif fg <= 55:
            classification = "😐 NEUTRAL"
            action = "⚪ Attendre signal clair"
        elif fg <= 75:
            classification = "😃 GREED"
            action = "🟡 Prudent pour VENTE"
        else:
            classification = "🤑 EXTREME GREED"
            action = "🔴 Risque de CORRECTION"
        
        print(f"{crypto:12} | F&G: {fg:3.0f}/100 | {classification} | {action}")

😱 Fear & Greed Index Analysis

BINANCECOIN  | F&G:  24/100 | 😱 EXTREME FEAR | 🟢 Opportunité d'ACHAT (contrarian)
BITCOIN      | F&G:  24/100 | 😱 EXTREME FEAR | 🟢 Opportunité d'ACHAT (contrarian)
CARDANO      | F&G:  24/100 | 😱 EXTREME FEAR | 🟢 Opportunité d'ACHAT (contrarian)
ETHEREUM     | F&G:  24/100 | 😱 EXTREME FEAR | 🟢 Opportunité d'ACHAT (contrarian)
SOLANA       | F&G:  24/100 | 😱 EXTREME FEAR | 🟢 Opportunité d'ACHAT (contrarian)


## 5. Signaux de Trading Combinés

In [11]:
# Calculer un score de signal pour chaque crypto
print("🎯 SIGNAUX DE TRADING COMBINÉS\n")
print("="*80)

signals = {}

for crypto, df in cryptos.items():
    score = 0
    reasons = []
    
    # RSI
    if 'rsi_14' in df.columns:
        rsi = df['rsi_14'].iloc[0]
        if rsi < 30:
            score += 2
            reasons.append("RSI sur-vendu (+2)")
        elif rsi > 70:
            score -= 2
            reasons.append("RSI sur-acheté (-2)")
    
    # MACD
    if 'macd_histogram' in df.columns:
        hist = df['macd_histogram'].iloc[0]
        if hist > 0:
            score += 1
            reasons.append("MACD haussier (+1)")
        else:
            score -= 1
            reasons.append("MACD baissier (-1)")
    
    # SMA Crossover
    if 'sma_crossover' in df.columns:
        if df['sma_crossover'].iloc[0] == 1:
            score += 2
            reasons.append("Golden Cross (+2)")
        else:
            score -= 2
            reasons.append("Death Cross (-2)")
    
    # Fear & Greed (contrarian)
    if 'fear_greed_index' in df.columns:
        fg = df['fear_greed_index'].iloc[0]
        if fg <= 25:
            score += 3
            reasons.append("Extreme Fear (+3)")
        elif fg > 75:
            score -= 3
            reasons.append("Extreme Greed (-3)")
    
    # Volume Spike
    if 'volume_spike' in df.columns:
        if df['volume_spike'].iloc[0] == 1:
            score += 1
            reasons.append("Volume spike (+1)")
    
    signals[crypto] = {'score': score, 'reasons': reasons}

# Trier par score
sorted_signals = sorted(signals.items(), key=lambda x: x[1]['score'], reverse=True)

for crypto, data in sorted_signals:
    score = data['score']
    reasons = data['reasons']
    
    # Déterminer signal global
    if score >= 5:
        signal = "🟢🟢🟢 FORT SIGNAL D'ACHAT"
    elif score >= 3:
        signal = "🟢🟢 SIGNAL D'ACHAT"
    elif score >= 1:
        signal = "🟢 Léger signal d'achat"
    elif score == 0:
        signal = "⚪ NEUTRE"
    elif score >= -2:
        signal = "🔴 Léger signal de vente"
    elif score >= -4:
        signal = "🔴🔴 SIGNAL DE VENTE"
    else:
        signal = "🔴🔴🔴 FORT SIGNAL DE VENTE"
    
    print(f"\n{crypto}")
    print(f"   Score: {score:+d} | {signal}")
    print(f"   Raisons:")
    for reason in reasons:
        print(f"      • {reason}")

🎯 SIGNAUX DE TRADING COMBINÉS


BINANCECOIN
   Score: +0 | ⚪ NEUTRE
   Raisons:
      • MACD baissier (-1)
      • Death Cross (-2)
      • Extreme Fear (+3)

BITCOIN
   Score: +0 | ⚪ NEUTRE
   Raisons:
      • MACD baissier (-1)
      • Death Cross (-2)
      • Extreme Fear (+3)

CARDANO
   Score: +0 | ⚪ NEUTRE
   Raisons:
      • MACD baissier (-1)
      • Death Cross (-2)
      • Extreme Fear (+3)

ETHEREUM
   Score: +0 | ⚪ NEUTRE
   Raisons:
      • MACD baissier (-1)
      • Death Cross (-2)
      • Extreme Fear (+3)

SOLANA
   Score: +0 | ⚪ NEUTRE
   Raisons:
      • MACD baissier (-1)
      • Death Cross (-2)
      • Extreme Fear (+3)


## 6. Statistiques Descriptives

In [12]:
# Statistiques sur les features numériques (quand on aura plus de données)
print("📊 Statistiques Descriptives (données actuelles limitées)\n")

# Pour l'instant, on a juste 1 ligne par crypto
# Mais préparons le code pour quand on aura des données historiques

numeric_cols = df_btc.select_dtypes(include=[np.number]).columns
print(f"Features numériques: {len(numeric_cols)}")
print(f"\nAperçu des valeurs actuelles (Bitcoin):")

# Sélectionner features importantes
important_features = ['price_usd', 'rsi_14', 'macd', 'sma_7', 'sma_30', 
                     'fear_greed_index', 'volume_24h_usd']

for feat in important_features:
    if feat in df_btc.columns:
        value = df_btc[feat].iloc[0]
        print(f"{feat:20} : {value:,.2f}")

📊 Statistiques Descriptives (données actuelles limitées)

Features numériques: 51

Aperçu des valeurs actuelles (Bitcoin):
price_usd            : 104,735.00
rsi_14               : 50.00
macd                 : 0.00
sma_7                : 104,735.00
sma_30               : 104,735.00
fear_greed_index     : 24.00
volume_24h_usd       : 63,767,851,128.36


## 7. Comparaison entre Cryptos

In [13]:
# Créer un DataFrame de comparaison
comparison_data = []

for crypto, df in cryptos.items():
    row = {
        'Crypto': crypto,
        'Prix': df['price_usd'].iloc[0] if 'price_usd' in df.columns else np.nan,
        'RSI': df['rsi_14'].iloc[0] if 'rsi_14' in df.columns else np.nan,
        'MACD': df['macd_histogram'].iloc[0] if 'macd_histogram' in df.columns else np.nan,
        'SMA_Cross': 'Golden' if df.get('sma_crossover', [0]).iloc[0] == 1 else 'Death',
        'F&G': df['fear_greed_index'].iloc[0] if 'fear_greed_index' in df.columns else np.nan,
        'Volume_Spike': 'Oui' if df.get('volume_spike', [0]).iloc[0] == 1 else 'Non',
        'Signal_Score': signals[crypto]['score']
    }
    comparison_data.append(row)

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('Signal_Score', ascending=False)

print("\n📊 TABLEAU COMPARATIF\n")
print(df_comparison.to_string(index=False))


📊 TABLEAU COMPARATIF

     Crypto      Prix   RSI  MACD SMA_Cross  F&G Volume_Spike  Signal_Score
BINANCECOIN    969.47 50.00  0.00     Death   24          Non             0
    BITCOIN 104735.00 50.00  0.00     Death   24          Non             0
    CARDANO      0.57 50.00  0.00     Death   24          Non             0
   ETHEREUM   3515.62 50.00  0.00     Death   24          Non             0
     SOLANA    159.19 50.00  0.00     Death   24          Non             0


## 8. Préparation pour le Machine Learning

In [14]:
# Features sélectionnées pour le ML (quand on aura + de données)
print("🤖 FEATURES POUR LE MACHINE LEARNING\n")
print("="*60)

ml_features = {
    'Prix & Volume': ['price_usd', 'volume_24h_usd', 'market_cap_usd'],
    'Indicateurs': ['rsi_14', 'macd', 'macd_histogram', 'sma_7', 'sma_30', 
                    'ema_12', 'ema_26', 'bb_upper', 'bb_lower', 'bb_width'],
    'Ratios': ['price_to_sma7_ratio', 'price_to_sma30_ratio', 'volume_ratio'],
    'Sentiment': ['fear_greed_index', 'fg_ma_7', 'fg_change_7d'],
    'Temporel': ['day_of_week', 'hour', 'is_weekend', 'month'],
    'Lag': ['price_usd_lag_1d', 'price_usd_lag_7d', 'volume_24h_usd_lag_1d'],
    'Binaires': ['sma_crossover', 'volume_spike', 'is_extreme_fear', 'is_extreme_greed']
}

total_features = sum(len(v) for v in ml_features.values())
print(f"Total features pour ML: {total_features}\n")

for category, features in ml_features.items():
    print(f"{category} ({len(features)}):")
    for feat in features:
        available = "✅" if feat in df_btc.columns else "❌"
        print(f"   {available} {feat}")
    print()

🤖 FEATURES POUR LE MACHINE LEARNING

Total features pour ML: 30

Prix & Volume (3):
   ✅ price_usd
   ✅ volume_24h_usd
   ✅ market_cap_usd

Indicateurs (10):
   ✅ rsi_14
   ✅ macd
   ✅ macd_histogram
   ✅ sma_7
   ✅ sma_30
   ✅ ema_12
   ✅ ema_26
   ✅ bb_upper
   ✅ bb_lower
   ✅ bb_width

Ratios (3):
   ✅ price_to_sma7_ratio
   ✅ price_to_sma30_ratio
   ✅ volume_ratio

Sentiment (3):
   ✅ fear_greed_index
   ✅ fg_ma_7
   ✅ fg_change_7d

Temporel (4):
   ✅ day_of_week
   ✅ hour
   ✅ is_weekend
   ✅ month

Lag (3):
   ✅ price_usd_lag_1d
   ✅ price_usd_lag_7d
   ✅ volume_24h_usd_lag_1d

Binaires (4):
   ✅ sma_crossover
   ✅ volume_spike
   ✅ is_extreme_fear
   ✅ is_extreme_greed



## 9. Insights et Conclusions

In [15]:
print("💡 INSIGHTS CLÉS\n")
print("="*80)

insights = [
    "1. DONNÉES ACTUELLES:",
    "   • 5 cryptos avec 54 features chacune",
    "   • Seulement 1 point de données (snapshot actuel)",
    f"   • Fear & Greed actuel: {df_btc['fear_greed_index'].iloc[0]:.0f} (Extreme Fear)",
    "",
    "2. SIGNAUX ACTUELS:",
]

# Top 3 opportunités
top3 = sorted_signals[:3]
insights.append("   Top 3 opportunités:")
for i, (crypto, data) in enumerate(top3, 1):
    insights.append(f"      {i}. {crypto} (score: {data['score']:+d})")

insights.extend([
    "",
    "3. PROCHAINES ÉTAPES:",
    "   ✅ Collecter données historiques Binance (ce soir)",
    "   ✅ Avoir 365 jours de données OHLCV",
    "   ✅ Re-générer features avec historique complet",
    "   ✅ Analyser corrélations entre features",
    "   ✅ Entraîner premier modèle ML",
    "",
    "4. MODÈLE ML PRÉVU:",
    "   • Target (y): Prix J+1 ou J+7",
    f"   • Features (X): ~35-40 features sélectionnées",
    "   • Modèles: Régression Linéaire, Random Forest, LSTM",
    "   • Validation: Train/Test split 80/20",
    "",
    "5. LIMITATIONS ACTUELLES:",
    "   ⚠️ Seulement 1 point de donnée par crypto",
    "   ⚠️ Impossible de calculer corrélations",
    "   ⚠️ Impossible d'entraîner modèle ML",
    "   ✅ RÉSOLU CE SOIR avec données Binance historiques",
])

for insight in insights:
    print(insight)

💡 INSIGHTS CLÉS

1. DONNÉES ACTUELLES:
   • 5 cryptos avec 54 features chacune
   • Seulement 1 point de données (snapshot actuel)
   • Fear & Greed actuel: 24 (Extreme Fear)

2. SIGNAUX ACTUELS:
   Top 3 opportunités:
      1. BINANCECOIN (score: +0)
      2. BITCOIN (score: +0)
      3. CARDANO (score: +0)

3. PROCHAINES ÉTAPES:
   ✅ Collecter données historiques Binance (ce soir)
   ✅ Avoir 365 jours de données OHLCV
   ✅ Re-générer features avec historique complet
   ✅ Analyser corrélations entre features
   ✅ Entraîner premier modèle ML

4. MODÈLE ML PRÉVU:
   • Target (y): Prix J+1 ou J+7
   • Features (X): ~35-40 features sélectionnées
   • Modèles: Régression Linéaire, Random Forest, LSTM
   • Validation: Train/Test split 80/20

5. LIMITATIONS ACTUELLES:
   ⚠️ Seulement 1 point de donnée par crypto
   ⚠️ Impossible de calculer corrélations
   ⚠️ Impossible d'entraîner modèle ML
   ✅ RÉSOLU CE SOIR avec données Binance historiques


## 10. Export pour Rapport

In [16]:
# Sauvegarder le tableau comparatif
output_dir = '../docs'
os.makedirs(output_dir, exist_ok=True)

output_file = f"{output_dir}/crypto_comparison_{datetime.now().strftime('%Y%m%d')}.csv"
df_comparison.to_csv(output_file, index=False)

print(f"✅ Tableau comparatif sauvegardé: {output_file}")
print("\n📊 Ce fichier peut être utilisé dans votre rapport M2")

✅ Tableau comparatif sauvegardé: ../docs/crypto_comparison_20251120.csv

📊 Ce fichier peut être utilisé dans votre rapport M2


---
## ✅ Conclusion

**État actuel:**
- ✅ 54 features créées par crypto
- ✅ Signaux de trading calculés
- ✅ Pipeline de feature engineering opérationnel

**Prochaine étape:**
- 🔄 Collecte Binance OHLCV (365 jours)
- 🔄 Re-analyse avec données complètes
- 🔄 Corrélations et visualisations
- 🔄 Premier modèle ML

**Date:** 12 novembre 2025  
**Auteur:** Karim Assi (@karimassi02)